<a href="https://colab.research.google.com/github/jinzzasol/Mathsage/blob/main/Mathsage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
os.chdir('drive/MyDrive/CS5624_NLP/MathSage')

In [4]:
!pip install -q transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.5 MB/s eta 0:00:00


In [5]:
import json
import pandas as pd
import glob

In [6]:
f = open('MathQA_DB/train.json')
data = json.load(f)
data[0]

{'Problem': "the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?",
 'Rationale': '"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"',
 'options': 'a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d ) rs . 350 , e ) none of these',
 'correct': 'a',
 'annotated_formula': 'divide(multiply(const_100, divide(multiply(36, const_100), multiply(3, 10))), multiply(3, 10))',
 'linear_formula': 'multiply(n2,const_100)|multiply(n0,n1)|divide(#0,#1)|multiply(#2,const_100)|divide(#3,#1)|',
 'category': 'gain'}

# Load Data
Create `pandas dataframe` from `JSON` files


In [7]:
mathqa_db = {}

def json_to_dataframe():
  json_paths = glob.glob('MathQA_DB/*.json')
  for json_path in json_paths:
    file_name = json_path.split('/')[-1].split('.')[0]  # Extract the file name without extension
    df = pd.read_json(json_path)
    mathqa_db[file_name] = df

In [8]:
json_to_dataframe()

In [9]:
mathqa_db.keys()

dict_keys(['dev', 'train', 'challenge_test', 'test'])

In [10]:
train_df = mathqa_db['train']
train_df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other


In [11]:
train_df[train_df.duplicated()]

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category


In [12]:
test_df = mathqa_db['test']
test_df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,a shopkeeper sold an article offering a discou...,"""giving no discount to customer implies sellin...","a ) 38 , b ) 27.675 , c ) 30 , d ) data inadeq...",a,"subtract(divide(multiply(add(const_100, 31.1),...","add(n1,const_100)|subtract(const_100,n0)|multi...",gain
1,what will be the difference between simple and...,"""s . i . = ( 1000 * 14 * 4 ) / 100 = rs . 560 ...","a ) 129 , b ) 130 , c ) 124 , d ) 133 , e ) 145",a,"subtract(subtract(multiply(1000, power(add(div...","divide(n0,const_100)|add(#0,const_1)|multiply(...",gain
2,there are 28 stations between hyderabad and ba...,"""the total number of stations = 30 from 30 sta...","a ) 156 , b ) 167 , c ) 870 , d ) 352 , e ) 380",c,"multiply(add(28, const_1), add(add(28, const_1...","add(n0,const_1)|add(#0,const_1)|multiply(#0,#1)|",physics
3,the present population of a town is 3888 . pop...,"""p = 3888 r = 20 % required population of town...","a ) 2500 , b ) 2100 , c ) 3500 , d ) 3600 , e ...",e,"add(3888, divide(multiply(3888, 20), const_100))","multiply(n0,n1)|divide(#0,const_100)|add(n0,#1)|",gain
4,the triplicate ratio of 1 : 9 is ?,"""13 : 93 = 1 : 729 answer : e""","a ) 1 : 0 , b ) 1 : 8 , c ) 1 : 7 , d ) 1 : 2 ...",e,"divide(power(const_2.0, 9), power(const_3.0, 9))","power(const_2.0,n1)|power(const_3.0,n1)|divide...",other


In [13]:
test_df[test_df.duplicated()]

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category


Make `answer` column selecting correct option in `options` column

**issue**
there are rows with double option, e.g., `a ) a ) 70 , b ) b ) 76 , c ) c ) 78 , d ) d )...`

In [14]:
def get_answer(data):
  answer = []
  for i in range(len(data)):
    str = data.loc[i,'options'].split(',')
    # print(str)
    for option in str:
      if (data.loc[i,'correct']) in option:
        # print("correct= ", data.loc[i, 'correct'])
        ans = option.split(')')[-1]
        break
    answer.append(ans)
    # print("ans= ", ans)
  return answer

# train_df

In [15]:
import numpy as np
answer = get_answer(train_df)
answer = np.array(answer)
answer

array([' rs . 400 ', ' 240 ', ' 270 ', ..., ' - 12 ', ' 242 ', ' nine '],
      dtype='<U56')

In [16]:
train_df['answer'] = answer
train_df

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category,answer
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain,rs . 400
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general,240
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general,270
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain,240 %
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other,1 / 2
...,...,...,...,...,...,...,...,...
29832,a man invests some money partly in 10 % stock ...,"""solution for an income of rs . 1 in 10 % stoc...","a ) 1 : 2 , b ) 3 : 5 , c ) 4 : 5 , d ) 16 : 1...",a,"divide(multiply(90, const_2), multiply(90, con...","multiply(n1,const_2)|multiply(n3,const_3)|divi...",other,1 : 2
29833,"average of 10 matches is 32 , how many runs on...","""explanation : average after 11 innings should...","a ) a ) 70 , b ) b ) 76 , c ) c ) 78 , d ) d )...",d,"subtract(multiply(add(32, 6), add(10, const_1)...","add(n1,n2)|add(n0,const_1)|multiply(n0,n1)|mul...",general,98
29834,solve below question 2 x + 1 = - 23,1 . subtract 1 from both sides : 2 x + 1 - 1 =...,"a ) - 8 , b ) - 9 , c ) - 12 , d ) - 4 , e ) 12",c,"divide(negate(add(23, 1)), 2)","add(n1,n2)|negate(#0)|divide(#1,n0)",general,- 12
29835,calculate 85184 ÷ ? = 352,"""answer let 85184 ÷ x = 242 then x = 85184 / 2...","a ) 241 , b ) 242 , c ) 244 , d ) 247 , e ) 240",b,"multiply(85184, 352)","multiply(n0,n1)|",general,242


In [17]:
# Split the *train* dataset into train, validation, test
from sklearn.model_selection import train_test_split

# 80% - train, 10% - validation, 10% - test
# First, create 20% of the test dataset, then split it into validation and test dataset
train, test = train_test_split(train_df, test_size=0.2, shuffle=False)
val, test = train_test_split(test, test_size=0.5, shuffle=True)

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

train.shape, val.shape, test.shape,

((23869, 8), (2984, 8), (2984, 8))

# Load MathBert pretrained model

Use MathBert from Jia Tracy Shen, Michiharu Yamashita, Ethan Prihar, Neil Heffernan, Xintao Wu, Ben Graff, Dongwon Lee (2023) "MathBERT: A Pre-trained Language Model for General NLP Tasks in Mathematics Education". . doi:[10.48550/arXiv.2106.07340](https://doi.org/10.48550/arXiv.2106.07340)

In [18]:
# Create new index
train_idx = [i for i in range(len(train.index))]
test_idx = [i for i in range(len(test.index))]
val_idx = [i for i in range(len(val.index))]

# Convert to numpy
x_train = train['Problem'].values[train_idx]
x_test = test['Problem'].values[test_idx]
x_val = val['Problem'].values[val_idx]

y_train = train['answer'].values[train_idx]
y_test = test['answer'].values[test_idx]
y_val = val['answer'].values[val_idx]


In [19]:
# Pytorch Version is available via Hugging Face Transformers
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

# Download the MathBERT -basevocab
# tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT")
# model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT")

# Download the MathBERT -mathvocab
tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT-custom")
model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT-custom")

Some weights of the model checkpoint at tbs17/MathBERT-custom were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
train_df['Problem'].iloc[-2]

'calculate 85184 ÷ ? = 352'

In [21]:
# Tokenize the problem text

# https://github.com/huggingface/transformers/issues/5480
# input = tokenizer(x_train[0], return_tensors='tf') # cannot feed this to model
input = tokenizer(train_df['Problem'].iloc[-2], return_tensors='pt')['input_ids']

In [22]:
input

tensor([[  101, 18422,  5594, 15136,  2549,  1099,  1029,  1027, 28906,   102]])

In [23]:
# tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(x_train[0])))
# tokens

In [24]:
# input_ids = tokenizer.encode(x_train[-2], return_tensors='pt')

# # Contextual Understanding
# # Get the model's output for contextual understanding
# output = model(input_ids)
output = model(input)

output

MaskedLMOutput(loss=None, logits=tensor([[[ 0.7762, -4.0762,  8.4563,  ...,  2.1553,  4.5549,  0.4989],
         [-1.4155, -2.2157,  3.5432,  ...,  0.2031, -1.8650,  1.9138],
         [ 0.0301, -4.7415,  2.0229,  ..., -0.7261, -3.1422,  0.1573],
         ...,
         [-4.3707, -1.5425,  1.7771,  ..., -2.3073, -0.2971, -3.2780],
         [-3.8644,  1.5295, -0.3828,  ..., -3.6289, -2.4256, -8.5556],
         [-1.6721, -0.3129,  2.6693,  ...,  2.4349,  2.4718,  1.2854]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [25]:
output.logits

tensor([[[ 0.7762, -4.0762,  8.4563,  ...,  2.1553,  4.5549,  0.4989],
         [-1.4155, -2.2157,  3.5432,  ...,  0.2031, -1.8650,  1.9138],
         [ 0.0301, -4.7415,  2.0229,  ..., -0.7261, -3.1422,  0.1573],
         ...,
         [-4.3707, -1.5425,  1.7771,  ..., -2.3073, -0.2971, -3.2780],
         [-3.8644,  1.5295, -0.3828,  ..., -3.6289, -2.4256, -8.5556],
         [-1.6721, -0.3129,  2.6693,  ...,  2.4349,  2.4718,  1.2854]]],
       grad_fn=<ViewBackward0>)

In [26]:
# The argument 2 specifies that the operation should be applied along the third axis
# of the tensor(axis numbering starts from 0), which is typically the axis representing the vocabulary.
output.logits.argmax(2)[0]

tensor([  101, 18422,  5594,  5594,  2549,  1099,  1029,  1027, 28906,   102])

In [27]:
# Interpretation and Explanation
# Extract the predicted tokens from the model's output
predicted_tokens = tokenizer.convert_ids_to_tokens(output.logits.argmax(2)[0])

# Convert the predicted tokens to text
predicted_text = ' '.join(predicted_tokens)

# Print the interpreted mathematical content
print("Mathematical Content:", predicted_text)

Mathematical Content: [CLS] calculate 85 85 ##4 ÷ ? = 352 [SEP]


In [28]:
# print(x_train[-2])

In [29]:
train_df['Rationale'][0]

'"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"'

In [30]:
# import tensorflow as tf
# input_tensor_tf = tf.convert_to_tensor(input)

In [31]:
# input_tensor_tf

In [32]:
output = model(input)
# output = model(input_tensor_tf)

In [33]:
output

MaskedLMOutput(loss=None, logits=tensor([[[ 0.7762, -4.0762,  8.4563,  ...,  2.1553,  4.5549,  0.4989],
         [-1.4155, -2.2157,  3.5432,  ...,  0.2031, -1.8650,  1.9138],
         [ 0.0301, -4.7415,  2.0229,  ..., -0.7261, -3.1422,  0.1573],
         ...,
         [-4.3707, -1.5425,  1.7771,  ..., -2.3073, -0.2971, -3.2780],
         [-3.8644,  1.5295, -0.3828,  ..., -3.6289, -2.4256, -8.5556],
         [-1.6721, -0.3129,  2.6693,  ...,  2.4349,  2.4718,  1.2854]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [34]:
# Get the predicted tokens from the model's output
predicted_tokens = tokenizer.convert_ids_to_tokens(output.logits.argmax(2)[0])
print(predicted_tokens)

# Initialize variables for equation extraction
equation = []
equations = []

# Iterate through the predicted tokens
for token in predicted_tokens:
    if token == '[SEP]':  # Detect the end of an equation
        if equation:
            equations.append(equation)
            equation = []  # Reset for the next equation
    else:
        equation.append(token)

# Print extracted equations
for eq_tokens in equations:
    equation_text = ' '.join(eq_tokens)
    print("Equation:", equation_text)


['[CLS]', 'calculate', '85', '85', '##4', '÷', '?', '=', '352', '[SEP]']
Equation: [CLS] calculate 85 85 ##4 ÷ ? = 352


In [35]:
# equations = [tokenizer.decode(input_ids[0][start:end+1], skip_special_tokens=True) for start, end in zip(start_positions, end_positions)]

# Print extracted equations
for equation in equations:
    print("Equation:", equation)

Equation: ['[CLS]', 'calculate', '85', '85', '##4', '÷', '?', '=', '352']


Another way to use pretrained MathBert

In [36]:
# from transformers import BertTokenizer, TFBertModel
# tokenizer = BertTokenizer.from_pretrained('tbs17/MathBERT',output_hidden_states=True)
# model = TFBertModel.from_pretrained("tbs17/MathBERT", from_pt=True) # add `from_pt=True` based on error message
# text = x_train[0]
# encoded_input = tokenizer(text, return_tensors='tf')['input_ids']
# output = model(encoded_input)

In [37]:
# output

Downstream work

In [38]:
import tensorflow as tf
from tensorflow import keras

formula_extraction_model = keras.Sequential([
    keras.layers.Input(shape=(output[0].shape[1],)),
    keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
formula_extraction_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
# Define your labeled data (X_train, y_train) for training
# X_train should be MathBERT embeddings
# y_train should be labels (1 for formula, 0 for non-formula)

formula_extraction_model.fit(input, 0, epochs=5, batch_size=32)

ValueError: ignored

In [42]:
# Assuming 'text' is the input text
input_embeddings = model(input)[0]
prediction = formula_extraction_model.predict(input_embeddings)

# 'prediction' will be a value close to 1 if a formula is detected and close to 0 otherwise
if prediction > 0.5:
    print("A mathematical formula is present in the text.")
else:
    print("No mathematical formula detected in the text.")


TypeError: ignored

In [ ]:
import tensorflow as tf
# Extract the relevant tensor from the output
embedding_tensor = output.last_hidden_state  # or any other relevant tensor you need

# Define your linear layer
linear_layer = tf.keras.layers.Dense(units=len(output))

# Pass the extracted tensor through the linear layer
linear_output = linear_layer(embedding_tensor)


In [ ]:
import torch
import torch.nn as nn

# Define a downstream model for formula extraction
class FormulaExtractionModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FormulaExtractionModel, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.fc(x)

In [ ]:
# Instantiate the downstream model
input_size = len(output)
num_classes = 2
formula_extraction_model = FormulaExtractionModel(input_size, num_classes)

# Define a sample input (MathBERT embeddings)
sample_input = output

# Pass the input through the downstream model
output = formula_extraction_model(linear_output)

# You can apply appropriate activation functions (e.g., softmax for classification) to interpret the output

In [ ]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf


tokenizer = BertTokenizer.from_pretrained('tbs17/MathBERT', output_hidden_states=True)
model = TFBertModel.from_pretrained("tbs17/MathBERT", from_pt=True)

def solve_math_problem(problem):
  """Solves a math problem in words using MathBERT.

  Args:
    problem: A string representing the math problem in words.

  Returns:
    A string representing the solution to the math problem.
  """

  # Tokenize the problem.
  tokens = tokenizer(problem, return_tensors='pt')['input_ids']

  tokens = tf.convert_to_tensor(tokens)

  # Encode the problem using MathBERT.
  encodings = model(tokens)

  # Get the output of MathBERT.
  output = model.predict(encodings)[0]

  # Decode the output of MathBERT.
  solution = tokenizer.decode(output.argmax(dim=-1))

  return solution

# Example usage:

problem = 'If you have 2 apples and 3 oranges, how many fruits do you have in total?'

solution = solve_math_problem(problem)

print(solution)
